# FinGPT-Forecaster Re-implemented with FinRobot

In this demo, we set up an agent to emulate the behavior of model in the fingpt-forecaster project with AutoGen, which takes a company's ticker symbol, recent basic financials and market news as input and predict its stock movements.

For detail of the original project, check out  [FinGPT-Forecaster](https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT_Forecaster)!  🔥[Demo](https://huggingface.co/spaces/FinGPT/FinGPT-Forecaster), [Medium Blog](https://medium.datadriveninvestor.com/introducing-fingpt-forecaster-the-future-of-robo-advisory-services-50add34e3d3c) & [Model](https://huggingface.co/FinGPT/fingpt-forecaster_dow30_llama2-7b_lora) on Huggingface🤗!

In [2]:
import autogen
from autogen.cache import Cache

from finrobot.utils import get_current_date, register_keys_from_json
from finrobot.data_source import FinnHubUtils, YFinanceUtils

After importing all the necessary packages and functions, we first instantiate a market analysis assistant and a user proxy using the agent classes defined by AutoGen. We also need the config for OpenAI & Finnhub here. 
- for openai configuration, rename OAI_CONFIG_LIST_sample to OAI_CONFIG_LIST and replace the api keys
- for finnhub configuration, rename config_api_keys_sample to config_api_keys and replace the api keys

In [3]:
# Read OpenAI API keys from a JSON file
config_list = autogen.config_list_from_json(
    "../OAI_CONFIG_LIST",
    filter_dict={"model": ["deepseek-chat"]},
)
llm_config = {"config_list": config_list, "timeout": 120, "temperature": 0}

# Register FINNHUB API keys
register_keys_from_json("../config_api_keys")

In [4]:
analyst = autogen.AssistantAgent(
    name="Market_Analyst",
    system_message="As a Market Analyst, one must possess strong analytical and problem-solving abilities, collect necessary financial information and aggregate them based on client's requirement."
    "For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get(
        "content", "").endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

Then we register our predefined functions with the userproxy.

Following our implementation with the fingpt-forecaster, we combined news/financials calls to the Finnhub API and stock data calls to the YFinance API.

In [5]:
from finrobot.toolkits import register_toolkits

tools = [
    {
        "function": FinnHubUtils.get_company_profile,
        "name": "get_company_profile",
        "description": "get a company's profile information"
    },
    {
        "function": FinnHubUtils.get_company_news,
        "name": "get_company_news",
        "description": "retrieve market news related to designated company"
    },
    {
        "function": FinnHubUtils.get_basic_financials,
        "name": "get_financial_basics",
        "description": "get latest financial basics for a designated company"
    },
    {
        "function": YFinanceUtils.get_stock_data,
        "name": "get_stock_data",
        "description": "retrieve stock price data for designated ticker symbol"
    }
]
register_toolkits(tools, analyst, user_proxy)

Start the conversation, asking the agent to call all the functions, and see how it aggregates all the information and leads to the conclusion.

In [8]:
# company = "Tesla"
company = "Tesla"

with Cache.disk() as cache:
    # start the conversation
    user_proxy.initiate_chat(
        analyst,
        message=f"Use all the tools provided to retrieve information available for {company} upon {get_current_date()}. Analyze the positive developments and potential concerns of {company} "
        "with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. "
        f"Then make a rough prediction (e.g. up/down by 2-3%) of the {company} stock price movement for next week. Provide a summary analysis to support your prediction.",
        cache=cache,
    )

User_Proxy (to Market_Analyst):

Use all the tools provided to retrieve information available for Tesla upon 2025-07-22. Analyze the positive developments and potential concerns of Tesla with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. Then make a rough prediction (e.g. up/down by 2-3%) of the Tesla stock price movement for next week. Provide a summary analysis to support your prediction.

--------------------------------------------------------------------------------
Market_Analyst (to User_Proxy):


***** Suggested tool call (call_0_e29f701d-38f2-493b-aecb-4f2f8a2f2294): get_company_profile *****
Arguments: 
{"symbol": "TSLA"}
**************************************************************************************************
***** Suggested tool call (call_1_4d2ca8ba-235a-4f06-89a9-16ce99b47ef5): get_company_news *****
Arguments: 
{"symbol": "TSLA", "start_date": "2025-07-15", "end_date": "2025-07-22", "max